<a href="https://colab.research.google.com/github/nbeaudoin/LangChain-Experimentation-Zone/blob/main/LangChain_Basics_Tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load dependencies

In [2]:
!pip -q install openai langchain huggingface_hub
!pip install python-dotenv

In [3]:
# prompt: load from drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv('/content/drive/MyDrive/secrets.json')

# Use variables
openai_api = os.getenv('OPENAI_API_KEY')
huggingface_api = os.getenv('HUGGINGFACE_API_KEY')

# Print variables
# print('Open API Key:', openai_api)
# print('HF API Key:', huggingface_api)


# Plain Conditional Generation
## First with OpenAI GPT 3

In [6]:
from langchain.llms import OpenAI

In [10]:
llm = OpenAI(
    model_name="text-davinci-003",
    temperature=0.9,
    max_tokens=100
)


In [11]:
text = "The quick brown fox jumps over the lazy dog"

print(llm(text))




The quick brown fox jumped over the lazy dog with ease. Even though the lazy dog was reluctant to move, the fox's agility allowed it to jump gracefully onto the other side.


## Now with T5-Flan-XL


In [12]:
from langchain.llms import HuggingFaceHub

In [23]:
llm_hf = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":0.9},
    huggingfacehub_api_token=huggingface_api
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [26]:
text = "Why did the chicken cross the road?"

print(llm_hf(text))

KeyboardInterrupt: ignored

## Prompt Templates

In [41]:
from langchain.prompts import prompt
from langchain import PromptTemplate

location_template = """
I want you to act as a naming consultant for new golf courses.

Return a list of golf course names. Each name should be short, catchy and easy to rememember.

What are some good names for a golf course that is in a {location_description}?
"""

prompt_template = PromptTemplate(
    input_variables=["location_description"],
    template=location_template
)


In [47]:
description = "a low-lying swam and wetland area with lots of wildlife"
description_02 = "a high mountain pass characterized by sights of birds"
description_03 = "an ocean-side landscape with crashing waves and sandy dunes"

# to see what the prompt will be like
prompt_template.format(location_description=description)

'\nI want you to act as a naming consultant for new golf courses.\n\nReturn a list of golf course names. Each name should be short, catchy and easy to rememember.\n\nWhat are some good names for a golf course that is in a a low-lying swam and wetland area with lots of wildlife?\n'

In [48]:
# querying the model with the prompt template
from langchain.chains import LLMChain

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

print(chain.run(description_02))




> Entering new LLMChain chain...
Prompt after formatting:

I want you to act as a naming consultant for new golf courses.

Return a list of golf course names. Each name should be short, catchy and easy to rememember.

What are some good names for a golf course that is in a a high mountain pass characterized by sights of birds?


> Finished chain.

1. Cranefall Pass Golf 
2. Aviary High Golf 
3. Peaks & Plumes Golf 
4. Cloudland Crest Golf 
5. High Flyer Golf 
6. Sky Summit Golf 
7. Feathery Fairways Golf 
8. Eagle’s Perch Golf 
9. Skyline Greens Golf 
10. Mountain Breeze Golf


## Few Shot Learning

In [51]:
from langchain import PromptTemplate, FewShotPromptTemplate

In [53]:
# First create the list of few show examples
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"}

]

In [54]:
# Specify the template to format the examples we did previously
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template
)

In [ ]:
# Finally, we creat the 'FewShotPromptTemplate' object
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt
    example_prompt=example_prompt,
    # The prefix is some text that goes before hte examplees in the prompt
    # Usually, this consists of instructions
    prefix = "Give the antonym of every inputs",
    # The suffix is some text that goes after teh examples in the prompt
    # Usually, this is where the user input will go
    suffix: "Word: {input}\nAntonym:",

)
